<a href="https://colab.research.google.com/github/niyati-rao/Capstone_Project_LWE/blob/main/LPN_Structured_Noise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Generating Instance

In [ ]:
import numpy as np

def generate_errors(m):
    while True:
        arr = np.random.randint(0, 2, m, dtype=np.uint8)
        if np.any(arr == 0):  # Ensure at least one 0
            return arr

eta = generate_errors(3)
print("eta:\n", eta)

def generate_instance_structured_noise(n, m, eta, u):

    A = np.random.randint(2, size=(m, n), dtype=np.uint8)
    b = np.dot(A, u) % 2

    b = (b + eta) % 2

    # print("Matrix A:\n", A)
    # print("Secret vector u:\n", u)
    # print("Vector b (with structured noise):\n", b)

    return A, b

u = np.random.randint(2, size=10, dtype=np.uint8)
A, b = generate_instance_structured_noise(10, 3, eta, u)

Y1 = (b[0] + np.dot(A[0], u)) % 2
Y2 = (b[1] + np.dot(A[1], u)) % 2
Y3 = (b[2] + np.dot(A[2], u)) % 2

print("Y1:\n", Y1)
print("Y2:\n", Y2)
print("Y3:\n", Y3)

eta:
 [1 1 0]
Y1:
 1
Y2:
 1
Y3:
 0


## Checking the Equations

In [ ]:
def eta_equations(A, b):

  for i in range(len(b)):
    print(b[i], " + ", end="(")
    for j in range(len(A[i])):
      if j == (len(A[i])-1):
        print(A[i][j], " * u_", j, end=")\n")
      else:
        print(A[i][j], " * u_", j, end=" + ")


n = 5
m = 3
u = np.random.randint(2, size=n, dtype=np.uint8)
# To represent a polynomial we are using an array with each term as a dictionary with degree and coefficient values
p = [{'x': 1, 'y': 1, 'z': 1, 'c': 1}]  # i.e., this is a polynomial with a single multinomial term 1*xyz
eta = generate_errors(m)
print("Eta: ", eta)

A, b = generate_instance_structured_noise(n, m, eta, u)

print("Roots of the polynomial in terms of A, b and u")
eta_equations(A, b)


Eta:  [1 0 0]
Roots of the polynomial in terms of A, b and u
1  + (1  * u_ 0 + 1  * u_ 1 + 1  * u_ 2 + 0  * u_ 3 + 1  * u_ 4)
1  + (1  * u_ 0 + 1  * u_ 1 + 0  * u_ 2 + 1  * u_ 3 + 0  * u_ 4)
0  + (1  * u_ 0 + 0  * u_ 1 + 0  * u_ 2 + 1  * u_ 3 + 0  * u_ 4)


## Define Polynomial Structure and Multiplication

In [ ]:
def multiply_terms(term1, term2):

    result = {'coeff': term1['coeff'] * term2['coeff']}

    for var in set(term1.keys()).union(term2.keys()):
        if var == 'coeff':
            continue
        power1 = term1.get(var, 0)  # Get the power of the variable in term1, default is 0
        power2 = term2.get(var, 0)  # Get the power of the variable in term2, default is 0
        result[var] = power1 + power2  # Add powers

    return result

def multiply_polynomials(poly1, poly2):

    result = []

    # Multiply each term in poly1 with each term in poly2
    for term1 in poly1:
        for term2 in poly2:
            new_term = multiply_terms(term1, term2)
            result.append(new_term)

    return result

poly1 = [{'coeff': 1}, {'b': 1, 'd': 1, 'coeff': 1}, {'e': 1, 'coeff': 1}]
poly2 = [{'coeff': 0}, {'a': 1, 'e': 1, 'coeff': 1}, {'b': 1, 'coeff': 1}, {'c': 1, 'coeff': 1}]

result = multiply_polynomials(poly1, poly2)

for term in result:
    print(term)

{'coeff': 0}
{'coeff': 1, 'e': 1, 'a': 1}
{'coeff': 1, 'b': 1}
{'coeff': 1, 'c': 1}
{'coeff': 0, 'b': 1, 'd': 1}
{'coeff': 1, 'd': 1, 'e': 1, 'b': 1, 'a': 1}
{'coeff': 1, 'b': 2, 'd': 1}
{'coeff': 1, 'b': 1, 'c': 1, 'd': 1}
{'coeff': 0, 'e': 1}
{'coeff': 1, 'e': 2, 'a': 1}
{'coeff': 1, 'e': 1, 'b': 1}
{'coeff': 1, 'e': 1, 'c': 1}


## Calculate the Final Equations

In [ ]:
def roots_equations(A, b):
    equations = []

    for i in range(len(b)):
        equation = []

        equation.append({"coeff": b[i]})

        for j in range(len(A[i])):
            if A[i][j] != 0:
                term = {"u_" + str(j): 1, "coeff": A[i][j]}
                equation.append(term)

        equations.append(equation)

    return equations

A = [
    [0, 1, 1, 0, 1],
    [1, 1, 1, 1, 0],
    [1, 0, 0, 1, 0]
]
b = [1, 0, 1]

equations = roots_equations(A, b)

for eq in equations:
    print(eq)

[{'coeff': 1}, {'u_1': 1, 'coeff': 1}, {'u_2': 1, 'coeff': 1}, {'u_4': 1, 'coeff': 1}]
[{'coeff': 0}, {'u_0': 1, 'coeff': 1}, {'u_1': 1, 'coeff': 1}, {'u_2': 1, 'coeff': 1}, {'u_3': 1, 'coeff': 1}]
[{'coeff': 1}, {'u_0': 1, 'coeff': 1}, {'u_3': 1, 'coeff': 1}]


## Multiply the Equations as p = xyz

In [ ]:
def multiply_equations(equations):
    result = equations[0]
    for equation in equations[1:]:
        result = multiply_polynomials(result, equation)
    return result

result = multiply_equations(equations)
print("\nResulting polynomial after multiplying all equations:")
print(result)


Resulting polynomial after multiplying all equations:
[{'coeff': 0}, {'coeff': 0, 'u_0': 1}, {'coeff': 0, 'u_3': 1}, {'coeff': 1, 'u_0': 1}, {'coeff': 1, 'u_0': 2}, {'coeff': 1, 'u_3': 1, 'u_0': 1}, {'coeff': 1, 'u_1': 1}, {'coeff': 1, 'u_1': 1, 'u_0': 1}, {'coeff': 1, 'u_3': 1, 'u_1': 1}, {'coeff': 1, 'u_2': 1}, {'coeff': 1, 'u_2': 1, 'u_0': 1}, {'coeff': 1, 'u_3': 1, 'u_2': 1}, {'coeff': 1, 'u_3': 1}, {'coeff': 1, 'u_3': 1, 'u_0': 1}, {'coeff': 1, 'u_3': 2}, {'coeff': 0, 'u_1': 1}, {'coeff': 0, 'u_1': 1, 'u_0': 1}, {'coeff': 0, 'u_3': 1, 'u_1': 1}, {'coeff': 1, 'u_1': 1, 'u_0': 1}, {'coeff': 1, 'u_1': 1, 'u_0': 2}, {'coeff': 1, 'u_3': 1, 'u_1': 1, 'u_0': 1}, {'coeff': 1, 'u_1': 2}, {'coeff': 1, 'u_1': 2, 'u_0': 1}, {'coeff': 1, 'u_3': 1, 'u_1': 2}, {'coeff': 1, 'u_1': 1, 'u_2': 1}, {'coeff': 1, 'u_1': 1, 'u_2': 1, 'u_0': 1}, {'coeff': 1, 'u_3': 1, 'u_1': 1, 'u_2': 1}, {'coeff': 1, 'u_3': 1, 'u_1': 1}, {'coeff': 1, 'u_3': 1, 'u_1': 1, 'u_0': 1}, {'coeff': 1, 'u_3': 2, 'u_1': 1}, {'co

## Make the Polynomial Multi-Linear

In [ ]:
def make_multilinear(polynomial):
    multilinear_poly = []
    for term in polynomial:
        if term['coeff'] == 0:
            continue

        new_term = {"coeff": term['coeff']}
        for var, power in term.items():
            if var != 'coeff':
                new_term[var] = 1  # no need to track higher powers in {0, 1} space

        multilinear_poly.append(new_term)

    return multilinear_poly

multilinear_polynomial = make_multilinear(result)

print("Multilinear polynomial:")
print(multilinear_polynomial)


Multilinear polynomial:
[{'coeff': 1, 'u_0': 1}, {'coeff': 1, 'u_0': 1}, {'coeff': 1, 'u_3': 1, 'u_0': 1}, {'coeff': 1, 'u_1': 1}, {'coeff': 1, 'u_1': 1, 'u_0': 1}, {'coeff': 1, 'u_3': 1, 'u_1': 1}, {'coeff': 1, 'u_2': 1}, {'coeff': 1, 'u_2': 1, 'u_0': 1}, {'coeff': 1, 'u_3': 1, 'u_2': 1}, {'coeff': 1, 'u_3': 1}, {'coeff': 1, 'u_3': 1, 'u_0': 1}, {'coeff': 1, 'u_3': 1}, {'coeff': 1, 'u_1': 1, 'u_0': 1}, {'coeff': 1, 'u_1': 1, 'u_0': 1}, {'coeff': 1, 'u_3': 1, 'u_1': 1, 'u_0': 1}, {'coeff': 1, 'u_1': 1}, {'coeff': 1, 'u_1': 1, 'u_0': 1}, {'coeff': 1, 'u_3': 1, 'u_1': 1}, {'coeff': 1, 'u_1': 1, 'u_2': 1}, {'coeff': 1, 'u_1': 1, 'u_2': 1, 'u_0': 1}, {'coeff': 1, 'u_3': 1, 'u_1': 1, 'u_2': 1}, {'coeff': 1, 'u_3': 1, 'u_1': 1}, {'coeff': 1, 'u_3': 1, 'u_1': 1, 'u_0': 1}, {'coeff': 1, 'u_3': 1, 'u_1': 1}, {'coeff': 1, 'u_2': 1, 'u_0': 1}, {'coeff': 1, 'u_2': 1, 'u_0': 1}, {'coeff': 1, 'u_3': 1, 'u_2': 1, 'u_0': 1}, {'coeff': 1, 'u_2': 1, 'u_1': 1}, {'coeff': 1, 'u_2': 1, 'u_1': 1, 'u_0': 1},

## Convert to Degree 1

In [ ]:
def convert_to_yi(polynomial):
    var_to_y_mapping = {}
    current_y_index = 0
    new_polynomial = []

    for term in polynomial:
        variables = tuple(sorted([var for var in term if var != 'coeff']))

        if len(variables) == 0:
            new_polynomial.append(term)
            continue

        if variables not in var_to_y_mapping:
            var_to_y_mapping[variables] = f'y_{current_y_index}'
            current_y_index += 1

        new_term = {'coeff': term['coeff'], var_to_y_mapping[variables]: 1}
        new_polynomial.append(new_term)

    return new_polynomial, var_to_y_mapping

converted_polynomial, y_mapping = convert_to_yi(multilinear_polynomial)

print("Converted polynomial:")
print(converted_polynomial)
print("\nMapping of variables to y_i:")
print(y_mapping)

Converted polynomial:
[{'coeff': 1, 'y_0': 1}, {'coeff': 1, 'y_0': 1}, {'coeff': 1, 'y_1': 1}, {'coeff': 1, 'y_2': 1}, {'coeff': 1, 'y_3': 1}, {'coeff': 1, 'y_4': 1}, {'coeff': 1, 'y_5': 1}, {'coeff': 1, 'y_6': 1}, {'coeff': 1, 'y_7': 1}, {'coeff': 1, 'y_8': 1}, {'coeff': 1, 'y_1': 1}, {'coeff': 1, 'y_8': 1}, {'coeff': 1, 'y_3': 1}, {'coeff': 1, 'y_3': 1}, {'coeff': 1, 'y_9': 1}, {'coeff': 1, 'y_2': 1}, {'coeff': 1, 'y_3': 1}, {'coeff': 1, 'y_4': 1}, {'coeff': 1, 'y_10': 1}, {'coeff': 1, 'y_11': 1}, {'coeff': 1, 'y_12': 1}, {'coeff': 1, 'y_4': 1}, {'coeff': 1, 'y_9': 1}, {'coeff': 1, 'y_4': 1}, {'coeff': 1, 'y_6': 1}, {'coeff': 1, 'y_6': 1}, {'coeff': 1, 'y_13': 1}, {'coeff': 1, 'y_10': 1}, {'coeff': 1, 'y_11': 1}, {'coeff': 1, 'y_12': 1}, {'coeff': 1, 'y_5': 1}, {'coeff': 1, 'y_6': 1}, {'coeff': 1, 'y_7': 1}, {'coeff': 1, 'y_7': 1}, {'coeff': 1, 'y_13': 1}, {'coeff': 1, 'y_7': 1}, {'coeff': 1, 'y_14': 1}, {'coeff': 1, 'y_14': 1}, {'coeff': 1, 'y_15': 1}, {'coeff': 1, 'y_16': 1}, {'coe

In [ ]:
def gf2_add(a, b):
    return a ^ b

def gf2_multiply(a, b):
    return a * b

def poly_to_matrix(polynomials, variables):
    matrix = []
    for poly in polynomials:
        row = [0] * len(variables)
        for term in poly:
            for var in term:
                if var != 'coeff':
                    idx = variables.index(var)
                    row[idx] = gf2_add(row[idx], term['coeff'])
        matrix.append(row)
    return matrix

def gaussian_elimination(matrix):
    n_rows = len(matrix)
    n_cols = len(matrix[0])

    for i in range(min(n_rows, n_cols)):
        if matrix[i][i] == 0:
            for j in range(i + 1, n_rows):
                if matrix[j][i] == 1:
                    matrix[i], matrix[j] = matrix[j], matrix[i]
                    break

        if matrix[i][i] == 1:
            for j in range(i + 1, n_rows):
                if matrix[j][i] == 1:
                    matrix[j] = [gf2_add(matrix[j][k], matrix[i][k]) for k in range(n_cols)]

    for i in range(min(n_rows, n_cols) - 1, -1, -1):
        if matrix[i][i] == 1:
            for j in range(i - 1, -1, -1):
                if matrix[j][i] == 1:
                    matrix[j] = [gf2_add(matrix[j][k], matrix[i][k]) for k in range(n_cols)]

    return matrix

In [ ]:
polys = [[{'coeff': 1, 'x': 1, 'y': 1, 'z':1}]]
mps = []
mappings = []
for p in polys:
  mps.append(make_multilinear(p))
conv, maps = [], []
for mp in mps:
  c, m = convert_to_yi(mp)
  conv.append(c)
  maps.append(m)
variables = []
for m in maps:
  variables.append(*m.values())
mat = poly_to_matrix(conv, variables)
result = gaussian_elimination(mat)

print("Result after Gaussian elimination:")
for row in result:
    print(row)

Result after Gaussian elimination:
[1]


In [ ]:
def convert_to_yi(polynomial, var_to_y_mapping=None, current_y_index=0):
    if var_to_y_mapping is None:
        var_to_y_mapping = {}

    new_polynomial = []

    for term in polynomial:
        variables = tuple(sorted([var for var in term if var != 'coeff']))

        if len(variables) == 0:
            new_polynomial.append(term)
            continue

        if variables not in var_to_y_mapping:

            var_to_y_mapping[variables] = f'y_{current_y_index}'
            current_y_index += 1

        new_term = {'coeff': term['coeff'], var_to_y_mapping[variables]: 1}
        new_polynomial.append(new_term)

    return new_polynomial, var_to_y_mapping, current_y_index

In [ ]:
import sympy as sp

N = 2**(5) - 1
u = np.random.randint(2, size=5, dtype=np.uint8)
eta = generate_errors(3)
y_map = {}
y_counter = 1
instances = []

for i in range(N+1):
  A, b = generate_instance_structured_noise(5, 3, eta, u)
  equations = roots_equations(A, b)
  result = multiply_all_equations(equations)
  multilinear_polynomial = make_multilinear(result)
  converted_polynomial, y_mapping, current_y_index = convert_to_yi(multilinear_polynomial, y_map, y_counter)
  instances.append(converted_polynomial)
  y_map.update(y_mapping)
  y_counter = current_y_index
  # print("Converted polynomial:")
  # print(converted_polynomial)

print("\nMapping of variables to y_i:")
print(y_map)

def convert_to_sympy(instances, y_map):
    sympy_eqs = []
    y_symbols = {key: sp.symbols(key) for key in y_map.values()}  # Map y_i's to SymPy symbols

    for eq in instances:
        sympy_expr = 0
        for term in eq:
            coeff = term['coeff']
            if len(term) == 1:  # Constant term
                sympy_expr += coeff
            else:
                product = coeff
                for y_var in term:
                    if y_var != 'coeff':
                        product *= y_symbols[y_var]
                sympy_expr += product
        sympy_eqs.append(sympy_expr)

    return sympy_eqs, y_symbols



sympy_eqs, y_symbols = convert_to_sympy(instances, y_map)
print("Y symbols", y_symbols.values())

for eq in sympy_eqs:
    print(eq)

solution = sp.solve(sympy_eqs, list(y_symbols.values()))
print("Solution:", solution)


Mapping of variables to y_i:
{('u_1',): 'y_1', ('u_0', 'u_1'): 'y_2', ('u_1', 'u_4'): 'y_3', ('u_2',): 'y_4', ('u_0', 'u_2'): 'y_5', ('u_2', 'u_4'): 'y_6', ('u_3',): 'y_7', ('u_0', 'u_3'): 'y_8', ('u_3', 'u_4'): 'y_9', ('u_1', 'u_2'): 'y_10', ('u_0', 'u_1', 'u_2'): 'y_11', ('u_1', 'u_2', 'u_4'): 'y_12', ('u_2', 'u_3'): 'y_13', ('u_0', 'u_2', 'u_3'): 'y_14', ('u_2', 'u_3', 'u_4'): 'y_15', ('u_0', 'u_1', 'u_4'): 'y_16', ('u_0', 'u_2', 'u_4'): 'y_17', ('u_0', 'u_4'): 'y_18', ('u_4',): 'y_19', ('u_1', 'u_3'): 'y_20', ('u_1', 'u_2', 'u_3'): 'y_21', ('u_1', 'u_3', 'u_4'): 'y_22', ('u_0', 'u_1', 'u_3'): 'y_23', ('u_0', 'u_3', 'u_4'): 'y_24', ('u_0',): 'y_25'}
Y symbols dict_values([y_1, y_2, y_3, y_4, y_5, y_6, y_7, y_8, y_9, y_10, y_11, y_12, y_13, y_14, y_15, y_16, y_17, y_18, y_19, y_20, y_21, y_22, y_23, y_24, y_25])
y_1 + y_10 + y_11 + y_12 + y_13 + y_14 + y_15 + y_2 + y_3 + 2*y_4 + 2*y_5 + 2*y_6 + y_7 + y_8 + y_9
y_1 + y_10 + y_11 + y_12 + y_16 + y_17 + y_2 + 3*y_3 + 2*y_6
y_12 + y_15 

# Sympy Implementation

**Assumptions:** The polynomial to which the oracle has access is defined with variables named $x_{i}$. See second cell.

In [ ]:
import sympy as sp
import numpy as np
import random
import math

In [ ]:
m = 2
x = sp.symbols(f'x0:{m}')
threshold = m//3
symmetric_polys = [sp.symmetric_poly(k, x) for k in range((threshold) + 1, m+1)]
P = sum(symmetric_polys)
P

x0*x1 + x0 + x1

In [ ]:
n = 10
m = 3
x = sp.symbols(f'x0:{m}')
threshold = m//3
symmetric_polys = [sp.symmetric_poly(k, x) for k in range((threshold) + 1, m+1)]
P = sum(symmetric_polys)
#x0, x1, x2 = sp.symbols('x0,x1,x2')
#P = (x0*x1*x2).as_poly(domain=sp.GF(2))

def oracle(s, n, m):
    a_vectors = [sp.Matrix([random.randint(0, 1) for _ in range(n)]) for _ in range(m)]
    b_values = []

    incorrect_indices = [random.randint(1,m) for i in range(threshold)]

    for i, a in enumerate(a_vectors):
        dot_product = (a.dot(s) % 2)
        error = 1 if i in incorrect_indices else 0
        b_values.append((dot_product + error) % 2)

    return list(zip(a_vectors, b_values))


In [ ]:
s = sp.Matrix([random.randint(0, 1) for _ in range(n)])
print(s)

Matrix([[1], [1], [1], [1], [0], [0], [1], [0], [1], [0]])


In [ ]:
equations = []
z = sp.Matrix([sp.symbols(f'z_{i}') for i in range(n)])
num_samples = 174
for _ in range(num_samples):
    instance = oracle(s, n, m)
    eqn = key_constraint(P, z, instance)
    equations.append(eqn)
#print(equations)

In [ ]:
linearization, map = linearize(equations)
print(linearization)
print(map)

[y_1 + y_112 + y_119 + y_120 + y_121 + y_122 + y_123 + y_124 + y_125 + y_126 + y_127 + y_128 + y_131 + y_133 + y_134 + y_135 + y_136 + y_137 + y_138 + y_139 + y_140 + y_141 + y_142 + y_143 + y_144 + y_147 + y_149 + y_151 + y_152 + y_154 + y_155 + y_156 + y_158 + y_159 + y_160 + y_162 + y_163 + y_165 + y_166 + y_167 + y_168 + y_169 + y_172 + y_173 + y_174 + y_18 + y_20 + y_22 + y_23 + y_24 + y_25 + y_26 + y_28 + y_29 + y_3 + y_30 + y_31 + y_33 + y_35 + y_4 + y_40 + y_41 + y_43 + y_44 + y_45 + y_46 + y_5 + y_55 + y_57 + y_58 + y_62 + y_63 + y_64 + y_68 + y_69 + y_71 + y_73 + y_75 + y_8 + y_80, y_0 + y_1 + y_103 + y_104 + y_105 + y_107 + y_108 + y_109 + y_111 + y_114 + y_115 + y_117 + y_118 + y_119 + y_120 + y_121 + y_125 + y_126 + y_127 + y_129 + y_13 + y_130 + y_131 + y_133 + y_134 + y_135 + y_14 + y_140 + y_147 + y_149 + y_150 + y_151 + y_155 + y_16 + y_161 + y_162 + y_165 + y_167 + y_168 + y_169 + y_17 + y_171 + y_172 + y_174 + y_2 + y_26 + y_27 + y_29 + y_30 + y_31 + y_32 + y_33 + y_

In [ ]:
len(linearization)

150

In [ ]:
sol = gaussian_elimination(linearization)
print(check_consistency(s, sol.args[0], map))
print(sol)

False
{(3931789778391794710779342214744778859499993448630289028275432669215977599084742771*y_174/89724275981703427882700406219263756397308775061326591976523746924532142282606824730, -16337482190133239913232273113608363083308823154510652535341462861832512405780938197*y_174/179448551963406855765400812438527512794617550122653183953047493849064284565213649460, -10150218914497914362502584331730360503273099680332651779312851541469393119641102417*y_174/11963236797560457051026720829235167519641170008176878930203166256604285637680909964, 41879179575404947930190810498084207007825783581967995167818766235373652562244849019*y_174/59816183987802285255133604146175837598205850040884394651015831283021428188404549820, 39301958981766806977079128630650702502849853130021865027323880935757064304966804917*y_174/89724275981703427882700406219263756397308775061326591976523746924532142282606824730, 13803040315109150379122344582010718565251082733178130906217386162247182010862499139*y_174/5981618398780228525513360

In [ ]:
s = sp.Matrix([random.randint(0, 1) for _ in range(n)])
num_samples = 100 # int(sum([math.comb(n,i) for i in range(3)])*math.pow(2,6))
z = sp.Matrix([sp.symbols(f'z_{i}') for i in range(n)])
equations = []
instance = oracle(s, n, m)
polynomial = P
for i in range(len(instance)):
    a_i, b_i = instance[i]
    rewritten_term = a_i.dot(z) + b_i
    polynomial = polynomial.subs(sp.Symbol(f'x{i}'), rewritten_term)
print(sp.expand(polynomial.as_expr()))

z_0**3 + 2*z_0**2*z_2 + z_0**2*z_3 + 2*z_0**2*z_6 + 2*z_0**2*z_7 + 4*z_0**2 + z_0*z_2**2 + z_0*z_2*z_3 + 2*z_0*z_2*z_6 + 3*z_0*z_2*z_7 + 6*z_0*z_2 + z_0*z_3*z_6 + z_0*z_3*z_7 + 3*z_0*z_3 + z_0*z_6**2 + 3*z_0*z_6*z_7 + 6*z_0*z_6 + z_0*z_7**2 + 5*z_0*z_7 + 2*z_0 + z_2**2*z_7 + 2*z_2**2 + z_2*z_3*z_7 + 2*z_2*z_3 + 2*z_2*z_6*z_7 + 4*z_2*z_6 + z_2*z_7**2 + 4*z_2*z_7 + 2*z_2 + z_3*z_6*z_7 + 2*z_3*z_6 + z_3*z_7 + z_3 + z_6**2*z_7 + 2*z_6**2 + z_6*z_7**2 + 4*z_6*z_7 + 2*z_6 + z_7**2 + z_7


In [ ]:
z

Matrix([
[z_0],
[z_1],
[z_2],
[z_3],
[z_4],
[z_5],
[z_6],
[z_7],
[z_8],
[z_9]])

In [ ]:
def key_constraint(polynomial, z, problem_instance):
    for i in range(len(problem_instance)):
        a_i, b_i = problem_instance[i]
        rewritten_term = a_i.dot(z) + b_i
        polynomial = polynomial.subs(sp.Symbol(f'x{i}'), rewritten_term)

    expanded = sp.expand(polynomial.as_expr())
    for z_i in z:
        expanded = expanded.replace(sp.Pow(z_i, sp.Wild('k')), z_i)

    expanded = sum((coef % 2) * term for term, coef in expanded.as_coefficients_dict().items())

    return expanded


In [ ]:
def linearize(expressions):
    monomials = sorted({m for expr in expressions for m in expr.as_expr().as_ordered_terms()}, key=str)
    monomial_to_var = {m: sp.Symbol(f'y_{i}') for i, m in enumerate(monomials)}
    converted_exprs = [expr.subs(monomial_to_var) for expr in expressions]
    return converted_exprs, monomial_to_var

In [ ]:
def substitute_solver(mapping, values):
    equations = [sp.Eq(lhs, rhs) for lhs, rhs in mapping.items()]
    equations += [sp.Eq(sp.Symbol(var), val) for var, val in values.items()]
    return sp.solve(equations, dict=True)

In [ ]:
def gaussian_elimination(constraints):
    y_vars = sorted(list({sym for constraint in constraints for sym in constraint.free_symbols if sym.name.startswith('y_')}), key=lambda x: int(x.name.split('_')[1]))
    system = sp.Matrix([[constraint.coeff(y) % 2 for y in y_vars] for constraint in constraints]) % 2
    rhs = sp.Matrix([-(constraint - sum(constraint.coeff(y) * y for y in y_vars)) % 2 for constraint in constraints])
    solution = sp.linsolve((system, rhs), *y_vars)

    return solution


In [ ]:
print(map)

{0: y_0, z_0: y_1, z_0*z_1: y_2, z_0*z_1*z_2: y_3, z_0*z_2: y_4, z_1: y_5, z_1*z_2: y_6, z_2: y_7}


**To Do**:


1.   Use a substituting function to extract what the secret key is guessed to be and verify whether it is the correct secret key.
2.   Iterate over several values of m and num_samples to see if correct value is ever achieved.



In [ ]:
def check_consistency(s, vector, mapping):
    z_vals = {f"z_{i}": s[i, 0] for i in range(s.rows)}
    y_vals = {}
    for z_expr, y in mapping.items():
        z_val = sp.expand(z_expr).subs(z_vals)
        if y in y_vals and y_vals[y] != z_val:
            return False
        y_vals[y] = z_val
    for i, term in enumerate(vector):
        if term != 0:
            evaluated = sp.expand(term).subs(y_vals) % 2
            if evaluated != y_vals.get(sp.Symbol(f'y_{i}'), 0):
                return False
    return True

In [ ]:
# u_0, u_1, u_2 = sp.symbols('u_0,u_1,u_2,u_3,u_4,u_5,u_6')
# polynomial = (u_0*u_1*u_2 + u_1*u_2*u_).as_poly(domain=sp.GF(2))
s = sp.Matrix([random.randint(0, 1) for _ in range(n)])
print("Secret key: ", s)
num_samples = 100 # int(sum([math.comb(n,i) for i in range(3)])*math.pow(2,6))
z = sp.Matrix([sp.symbols(f'z_{i}') for i in range(n)])
equations = []
for _ in range(num_samples):
    instance = oracle(s, n, m)
    eqn = key_constraint(P, z, instance)
    equations.append(eqn)
#print(equations)
linearization, map = linearize(equations)
sol = gaussian_elimination(linearization)
print(check_consistency(s, sol.args[0], map))
print(sol)

False

In [ ]:
m = 2
for i in range ((m//3)+1, m+1):
  print(i)

1
2
